#### Import Dependencies

In [ ]:
import yaml
from jinja2 import Template
from langsmith import Client

#### RAG Pipeline

In [ ]:
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [ ]:
preprocessed_context = "- a \n- b"
question = "What is a?"

In [ ]:
prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.

Context:
{preprocessed_context}

Question:
{question}
"""

In [ ]:
print(prompt)

#### Jinja Templates

In [ ]:
jinja_template = """
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.

Context:
{{ preprocessed_context }}

Question:
{{ question }}
"""

In [ ]:
template = Template(jinja_template)

In [ ]:
rendered_prompt = template.render(preprocessed_context=preprocessed_context, question=question)

In [ ]:
print(rendered_prompt)

In [ ]:
def build_prompt_jinja(preprocessed_context, question):

    prompt = """
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.

Context:
{{ preprocessed_context }}

Question:
{{ question }}
"""

    template = Template(prompt)
    rendered_prompt = template.render(
        preprocessed_context=preprocessed_context, 
        question=question
    )

    return rendered_prompt

In [ ]:
print(build_prompt_jinja(preprocessed_context, question))

In [ ]:
def prompt_template_config(yaml_file, prompt_key):

    with open(yaml_file, 'r') as file:
        config = yaml.safe_load(file)

    template_content = config['prompts'][prompt_key]

    template = Template(template_content)

    return template


In [ ]:
def build_prompt_jinja(preprocessed_context, question):

    template = prompt_template_config("prompts/retrieval_generation.yaml", "retrieval_generation")
    
    rendered_prompt = template.render(
        preprocessed_context=preprocessed_context, 
        question=question
    )

    return rendered_prompt

In [ ]:
print(build_prompt_jinja(preprocessed_context, question))

#### Prompt Registries

In [ ]:
ls_client = Client()

In [ ]:
ls_template = ls_client.pull_prompt("retrieval-generation")

In [ ]:
ls_template

In [ ]:
print(ls_template.messages[0].prompt.template)

In [ ]:
def prompt_template_registry(prompt_name):

    template_content = ls_client.pull_prompt(prompt_name).messages[0].prompt.template

    template = Template(template_content)

    return template


In [ ]:
print(prompt_template_registry("retrieval-generation").render(preprocessed_context=preprocessed_context, question=question))